<a href="https://colab.research.google.com/github/EvansXGames/CMP405-Project/blob/main/CMP405_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier

# === Load & sample data ===
df = pd.read_csv("games.csv")
df['blueWins'] = df['winner'].apply(lambda x: 1 if x == 1 else 0)
df = df.drop(columns=[col for col in ['gameId', 'creationTime', 'seasonId', 'gameDuration', 'winner'] if col in df.columns])
df = df.dropna()
df = df.sample(frac=0.3, random_state=42)  # Speed up training

X = df.drop(columns=['blueWins'])
y = df['blueWins']

# === Train model ===
model = RandomForestClassifier(n_estimators=20, random_state=42)
model.fit(X, y)

# === Save model and features ===
joblib.dump(model, 'lol_win_predictor.pkl')
joblib.dump(X.columns.tolist(), 'model_features.pkl')

# === AI Agent ===
def predict_blue_win(game_features: dict, threshold=0.5):
    model = joblib.load('lol_win_predictor.pkl')
    feature_list = joblib.load('model_features.pkl')
    input_data = {feature: game_features.get(feature, 0) for feature in feature_list}
    input_df = pd.DataFrame([input_data])
    proba = model.predict_proba(input_df)[0][1]
    return "Blue Win" if proba >= threshold else "Red Win"